$\LaTeX \text{ commands here}
\newcommand{\R}{\mathbb{R}}
\newcommand{\im}{\text{im}\,}
\newcommand{\norm}[1]{||#1||}
\newcommand{\inner}[1]{\langle #1 \rangle}
\newcommand{\span}{\mathrm{span}}
\newcommand{\proj}{\mathrm{proj}}
\newcommand{\OPT}{\mathrm{OPT}}
$

<hr style="border: 5px solid black">

**Georgia Tech, CS 4540**

# L18: Online Learning for Solving Games and LPs

Kevin Lai

*Thursday, October 31, 2019*

## Approximate Nash Equilibria

**Theorem** (Min-max): For any matrix $A \in \R^{n \times m}$ we have
$$\min_{p \in \Delta_n} \max_{q \in \Delta_m} p^\top A q = \lambda^* = \max_{q \in \Delta_m} \min_{p \in \Delta_n} p^\top A q$$

$\lambda^*$ is the _value_ of the game

A pair $(\mathbf{\hat{x}},\mathbf{\hat{y}})\in \Delta_n \times \Delta_m$ is an **$\epsilon$-approximate Nash Equilibrium** for a game $A\in \R^{n\times m}$ if:
$$ \lambda^* - \epsilon \le \mathbf{\hat{x}}^\top A \mathbf{\hat{y}} \le \lambda^* + \epsilon $$

## Finding approximate Nash Equilibria

Algorithm: **"Randomized Weighted Majority vs. Best Response"**
- Input: matrix $A\in [0,1]^{n\times m}$
- Initialize: $\mathbf{x}_{1} = (\frac{1}{n}, ..., \frac{1}{n})$
- For $t = 1, ..., T$
 - $\forall i, \mathbf{x}_{t}(i) = \frac{\mathbf{x}_{t-1}(i) \exp \left(-\eta A_i \mathbf{y}_{t-1} \right)}{z_{t-1}}$ where $z_{t-1}$ normalizes (Randomized Weighted Majority)
 - $\mathbf{y}_{t} = \max_{y\in\Delta_m} \mathbf{x}_t^\top A \mathbf{y}$ (best response)
- Output: $\mathbf{\bar{x}}_T = \frac{1}{T} \sum_{t=1}^T \mathbf{x}_t$


Recall that Randomized Weighted Majority algorithm has the following guarantee for any set of convex functions $f_t$ with $\norm{\nabla_t}_\infty \le G_\infty$  over the simplex:
$$
\text{Regret}_T(RWM) = \sum_{t=1}^T f_t(\mathbf{x}_t) - \min_{\mathbf{x} \in K} \sum_{t=1}^T f_t(\mathbf{x}) \leq 2G_\infty\sqrt{2T\log n}
$$

**Lemma**: Randomized Weighted Majority vs. Best Response outputs a $2\sqrt{2\log n/T}$-approximate solution to the zero-sum game

**Proof**:
\begin{align}
\frac{1}{T} \sum_{t} \mathbf{x}_t^\top A \mathbf{y}_t&\le \frac{1}{T} \min_{\mathbf{x} \in \Delta_n} \sum_{t} \mathbf{x}^\top A \mathbf{y}_t + 2\sqrt{2\log n/T}\\
&= \min_{\mathbf{x} \in \Delta_n} \mathbf{x} A \mathbf{\bar{y}}_T + 2\sqrt{2\log n/T}\\
&\le \max_{\mathbf{y}\in \Delta_m} \min_{\mathbf{x} \in \Delta_n} \mathbf{x}^\top A \mathbf{y} + 2\sqrt{2\log n/T}\\
&= \lambda^* + 2\sqrt{2\log n/T}
\end{align}

\begin{align}
\frac{1}{T} \sum_{t} \mathbf{x}_t A \mathbf{y}_t &\ge \frac{1}{T} \sum_{t} \mathbf{x}_t A \mathbf{y}^*\\
&=  \max_{\mathbf{y}\in \Delta_m} \mathbf{\bar{x}}^\top A y\\
&\ge \min_{\mathbf{x} \in \Delta_n} \max_{\mathbf{y}\in \Delta_m} \mathbf{x}^\top A \mathbf{y} \\
&= \lambda^*
\end{align}

### Changing RWM to OGD

Recall that Online Gradient Descent (OGD) has the following guarantee for any set of $G$-Lipschitz convex functions $f_t$ over a compact convex set $\mathcal{X}$ with diameter $D$:
$$
\text{Regret}_T(OGD) = \sum_{t=1}^T f_t(\mathbf{x}_t) - \min_{\mathbf{x} \in K} \sum_{t=1}^T f_t(\mathbf{x}) \leq \frac 3 2 GD\sqrt{T}
$$
#### Problem
If we use OGD where $f_t(\mathbf{x}) = \mathbf{x}^\top A\mathbf{y}_{t-1}$ and $K = \Delta_n$, how good of an approximation is the output of "OGD vs. Best Response" after $T$ iterations in terms of $G$ and $D$?

**Bonus**: Solve for $G$ and $D$ in this specific case

For reference, here's part of the proof from "RWM vs. BR":
\begin{align}
\frac{1}{T} \sum_{t} \mathbf{x}_t^\top A \mathbf{y}_t&\le \frac{1}{T} \min_{\mathbf{x} \in \Delta_n} \sum_{t} \mathbf{x}^\top A \mathbf{y}_t + 2\sqrt{2\log n/T}\\
&= \min_{\mathbf{x} \in \Delta_n} \mathbf{x} A \mathbf{\bar{y}}_T + 2\sqrt{2\log n/T}\\
&\le \max_{y\in \Delta_m} \min_{\mathbf{x} \in \Delta_n} \mathbf{x}^\top A \mathbf{y} + 2\sqrt{2\log n/T}\\
&= \lambda^* + 2\sqrt{2\log n/T}
\end{align}

### Changing BR to OGD

#### Problem
Now suppose we use OGD instead of Best Response. In other words, let $h_t(\mathbf{y}) = \mathbf{x}_t^\top A\mathbf{y}$ and $K = \Delta_n$. How good of an approximation is the output of "OGD vs. OGD" after $T$ iterations in terms of $G$ and $D$?

For reference, here's part of the proof from "RWM vs. BR":
\begin{align}
\frac{1}{T} \sum_{t} \mathbf{x}_t^\top A \mathbf{y}_t &\ge \frac{1}{T} \sum_{t} \mathbf{x}_t^\top A \mathbf{y}^*\\
&=  \max_{\mathbf{y}\in \Delta_m} \mathbf{\bar{x}}^\top A y\\
&\ge \min_{\mathbf{x} \in \Delta_n} \max_{y\in \Delta_m} \mathbf{x}^\top A \mathbf{y} \\
&= \lambda^*
\end{align}

## Approximate Nash Equilibrium through No-Regret algorithms

Algorithm: **"No-regret algorithm vs. No-regret algorithm**
- Input: matrix $A\in \R^{n\times m}$
- Initialize: $\mathbf{x}_{1} = (\frac{1}{n}, ..., \frac{1}{n})$
- For $t = 1, ..., T$
 - Update $\mathbf{x}_t$ according to no-regret algorithm 1 given loss function $f_t(\mathbf{x}) = \mathbf{x}^\top A\mathbf{y}_{t-1}$
 - Update $\mathbf{y}_t$ according to no-regret algorithm 1 given loss function $h_t(\mathbf{y}) = \mathbf{x}_{t-1} A^\top \mathbf{y}$
- Output: $(\mathbf{\bar{x}}_T, \mathbf{\bar{y}}_T) = (\frac{1}{T} \sum_{t=1}^T \mathbf{x}_t, \frac{1}{T} \sum_{t=1}^T \mathbf{y}_t)$

**Theorem**: Given two no-regret algorithms $\mathcal{A}_1$ and $\mathcal{A}_2$ whose average regret is bounded by $R_1^T$ and $R_2^T$ respectively after $T$ iterations, then if we run "$\mathcal{A}_1$ vs. $\mathcal{A}_2$", the output $(\mathbf{\bar{x}}_T, \mathbf{\bar{y}}_T)$ is an $(R_1^T + R_2^T)$-approximate Nash Equilibrium.

**Theorem**: Given two no-regret algorithms $\mathcal{A}_1$ and $\mathcal{A}_2$ whose average regret is bounded by $R_1^T$ and $R_2^T$ respectively after $T$ iterations, then if we run "$\mathcal{A}_1$ vs. $\mathcal{A}_2$", the output $(\mathbf{\bar{x}}_T, \mathbf{\bar{y}}_T)$ is an $(R_1^T + R_2^T)$-approximate Nash Equilibrium.

This theorem works for general convex-concave games $g(\mathbf{x},\mathbf{y})$ for $\mathbf{x}\in \mathcal{X}$ and $\mathbf{y} \in \mathcal{Y}$.
    

## LP Optimality through LP Feasibility

Suppose we want to solve an LP, call it "LP1":
\begin{align}
\min_{\mathbf{x}} \mathbf{c}^\top \mathbf{x}\\
A\mathbf{x} \le b\\
\mathbf{x} \ge 0
\end{align}

However, we only have access to an oracle $\mathcal{A}$ that solves LP feasibility, i.e. given $\tilde{A}$ and $\tilde{b}$, the oracle outputs an $\mathbf{x}$ such that $\tilde{A} \mathbf{x} \le b$ and $\mathbf{x} \ge 0$ if such an $\mathbf{x}$ exists, otherwise it outputs "False".

#### Problem

Suppose we know that LP1 has optimum value $\alpha$. How many calls to $\mathcal{A}$ do we need to compute an $\mathbf{x}$ that satisfies the constraints and is optimal for LP1?

Now suppose LP1 has optimum value in the range $[\alpha,\beta]$.  How many calls to $\mathcal{A}$ do we need to compute an $\mathbf{x}$ that satisfies the constraints and is within $\epsilon$ optimal for LP1?

## Using Perceptron to solve LPs

Recall the Perceptron algorithm guarantee:
Let $S$ be a sequence of labeled examples consistent with a linear threshold function i.e. $\forall \mathbf{x}\in S, \mathbf{w}^* \cdot \mathbf{x} > 0 $. Then after $O(1/\gamma^2)$ iterations, the perceptron algorithm will output a $\mathbf{w}$ such that $\mathbf{w}\cdot \mathbf{x} > 0$ for all examples $\mathbf{x}$, where
$$\gamma = \min_{\mathbf{x}\in S} \frac{|\mathbf{w}^* \cdot \mathbf{x}|}{\norm{\mathbf{w}^*}\cdot\norm{\mathbf{x}}}$$

#### Problem
Suppose we want to determine feasibility for the following LP, call it "LP2":
\begin{align}
\exists? x: A\mathbf{x} \ge 0
\end{align}
Let $x^*$ be a feasible solution to LP2. Suppose that $\mathbf{x}^*$ has a margin $\gamma > 0$ in the following sense:
$$\gamma = \min_{i} \frac{|\mathbf{x}^* \cdot A_{i,:}|}{\norm{\mathbf{x^*}}\cdot \norm{A_{i,:}}}$$
Then show how to use the Perceptron algorithm to find a feasible solution to LP2 in $O(1/\gamma^2)$ iterations.

**Hard Bonus**: Show how to solve LP1 using the Perceptron algorithm. Make any margin assumptions you need to make.